In [1]:
import os
import sys
import subprocess
import shutil
import pandas as pd
import numpy as np
import xmltodict
import json
import hashlib
import requests
import getpass
from requests import Response
from time import sleep

import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import table

HOMEDIR="."

In [2]:
sys.path.append("../../lib")
print(sys.path)
from pyutils import *
from pdutils import *
from pdpltutils import *
from gputils import *
from iputils import *

['/home/jblake1/git/LELPerformance/Network_Measurement_Project/Jupyter', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/jblake1/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '../../lib']


In [3]:
DIGIIP="192.168.2.1"
DIGIPW="Cmu5@ty@l@60"
DIGI_TESTCMD="config network modem wwan1 sim_slot"

In [7]:
basecmd = f"sshpass -p {DIGIPW} ssh admin@{DIGIIP}"
cmd=f"{basecmd} {DIGI_TESTCMD}"

def runDIGIcmd(icmd, output = True):
    cmd=f"{basecmd} {icmd}"
    return cmd_all(cmd, output=output)
runDIGIcmd(DIGI_TESTCMD)


STDOUT
2

STDERR


{'stdout': ['2', ''], 'stderr': []}

In [5]:
print(cmd)

sshpass -p Cmu5@ty@l@60 ssh admin@192.168.2.1 config network modem wwan1 sim_slot


In [6]:
oscmd(cmd)

2


0

In [8]:
runDIGIcmd(DIGI_TESTCMD + " 2",output=False)

{'stdout': [''], 'stderr': []}

In [30]:
def getModemStatus(output=True):
    res = runDIGIcmd("show modem verbose",output=output)
    modemstat = None
    stdoutclean = [line.strip() for line in res['stdout'] if line.startswith(" Modem") or line.startswith(" wwan1")]
    if len(stdoutclean) < 2:
#         print("Modem is not connected")
        modemstat = 'not_connected'
    elif 'disabl' in stdoutclean[1]:
#         print("Modem is disabled")
        modemstat = 'disabled'
    elif 'found' in stdoutclean[1]:
#         print("Modem is not found")
        modemstat = 'not_found'
    elif 'enabling' in stdoutclean[1]:
#         print("Modem is being enabled")
        modemstat = 'enabling'
    else:
        data = [fld for fld in stdoutclean[1].split(' ') if fld != '']
        dbm = data[-2].replace("(","")
        modemstat = f"{data[3]}_SIM:{data[1]}_DBM:{dbm}"
        if output == True: print(data)
    return modemstat,stdoutclean


modemstat, output = getModemStatus(output=False)
print(modemstat)

connecting_SIM:2_DBM:-39


In [31]:
def toggleSIM():    
    res = runDIGIcmd(DIGI_TESTCMD,output=False)
    curSIM = res['stdout'][0]
    newSIM = '1' if curSIM == '2' else '2'
    print(curSIM, newSIM)
    runDIGIcmd(DIGI_TESTCMD + " " + newSIM,output=False)

In [32]:
toggleSIM()

2 1


In [33]:
TIMEOUTCT=40
TIMEOUTWT=5

def waitForConnect():
    for ii in range(0,TIMEOUTCT):
        mstat, output = getModemStatus(output=False)
        nonstatelst = ['not_connected','not_found','disabled','enabling','disabling']
        if (mstat not in nonstatelst):
            print(f"{ii} {mstat} {output}")
            break
        else:
            print(f"{ii} {mstat} {output}")
        sleep(TIMEOUTWT)

waitForConnect()

0 disabled ['Modem  SIM        Status     APN  Signal Strength', 'wwan1  2 (ready)  disabling       Good (-82 dBm)']
1 not_connected ['Modem not available.']
2 disabled ['Modem  SIM        Status    APN  Signal Strength', 'wwan1  1 (ready)  disabled       Good (-81 dBm)']
3 not_connected ['Modem not available.']
4 not_connected ['Modem not available.']
5 not_connected ['Modem not available.']
6 not_connected ['Modem not available.']
7 not_connected ['Modem not available.']
8 not_connected ['Modem not available.']
9 not_connected ['Modem not available.']
10 not_connected ['Modem not available.']
11 not_connected ['Modem not available.']
12 not_connected ['Modem not available.']
13 not_connected ['Modem not available.']
14 not_found ['Modem  SIM            Status  APN  Signal Strength', 'wwan1  1 (Not found)  failed       Excellent (-62 dBm)']
15 not_found ['Modem  SIM            Status  APN  Signal Strength', 'wwan1  1 (Not found)  failed       Excellent (-62 dBm)']
16 not_connected ['M

In [ ]:
_ = getModemStatus(output=False)

In [ ]:
_ = getModemStatus(output=False)

## Remote Manager Web API

In [ ]:
#!python3
# Python example using requests library 
# Use 'pip install requests' to install the requests library.           

host="remotemanager.digi.com"            
api="https://%s/ws/v1/devices/inventory/00000000-00000000-0004F3FF-FF94F583/" % host

# user = input("Username for %s: " % host)
user="jblake1@andrew.cmu.edu"
# password = getpass.getpass("Password for user %s at %s:" % (user, host))
password="Cmu5@ty@l@60"

# Retaining a session when doing multiple requests can speed up your requests
session = requests.Session()
session.auth = (user, password)

rsp = session.get(api)
rsp.raise_for_status()
print("Status code:", rsp.status_code)
# print("Result text:", rsp.text)

stts=json.loads(rsp.text)

for xx in stts.keys():
    print(f"{xx} -> {stts[xx]}")

In [ ]:

KEYLST = ['connection_status','provider','apn','carrier','last_connect','last_disconnect']
def checkStatus(output=False,keylst=KEYLST):
    session = requests.Session()
    session.auth = (user, password)
    rsp = session.get(api)
    rsp.raise_for_status()
#     print(rsp.text)
    stts=json.loads(rsp.text)
    if (output):
        keyresult = " ".join([stts[key] for key in keylst if key in stts ])
        print(keyresult)
    del rsp
    del session
    return stts

TIMEOUTCT=20
TIMEOUTWT=2

def waitForConnect():
    for ii in range(0,TIMEOUTCT):
        stts = checkStatus()
        connectstat=stts['connection_status']
        carrier = stts['provider']
        if (connectstat=='connected'):
            print(f"Connected to {carrier} last_disconnect={stts['last_disconnect']}")
#             break
        print(f"{ii}")
        sleep(TIMEOUTWT)
                  
# waitForConnect()

In [ ]:
checkStatus(output=True)

## Local Web API

In [ ]:
#!python3
# Python example using requests library 
# Use 'pip install requests' to install the requests library.           

host="192.168.2.1"            
api="https://%s/cgi-bin/config.cgi/value/service/ssh" % host
print(api)
# https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh

# user = input("Username for %s: " % host)
user="admin"
# password = getpass.getpass("Password for user %s at %s:" % (user, host))
password="Cmu5@ty@l@60"

# # Retaining a session when doing multiple requests can speed up your requests
# session = requests.Session()
# session.auth = (user, password)

# rsp = session.get(api)
# rsp.raise_for_status()
# print("Status code:", rsp.status_code)
# # print("Result text:", rsp.text)

cmd=f"curl -k -u {user}:{password} {api}"
print(cmd)
response = cmd_all(cmd, output=True)

# stts=json.loads(rsp.text)

# for xx in stts.keys():
#     print(f"{xx} -> {stts[xx]}")

In [ ]:
# config network modem wwan1 sim_slot
api="https://%s/cgi-bin/config.cgi/value/network/interface/wwan1" % host
cmd=f"curl -k -u {user}:{password} {api} -X GET"
print(cmd)
response = cmd_all(cmd, output=True)

In [ ]:
cmd="curl -k -u admin:Cmu5@ty@l@60 https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh"
cmd_all(cmd,output=True)